### 导入需要的库

In [1]:
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda, LSTM, RepeatVector, TimeDistributed, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
from scipy import spatial
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import os

Using TensorFlow backend.


# 加载目录和文档
首先，我们将设置主目录和一些有关文本特征的变量。我们将序列长度设置为5-40，将词汇表中的最大单词数设置为93250(93254)，我们将使用300维embeddings。最后，从txt加载文本。文本文件来源于人民日报，包含大约10万个句子。

In [2]:
BASE_DIR = './data/'
TRAIN_DATA_FILE = BASE_DIR + 'train.txt'# 10+万条问句
GLOVE_EMBEDDING = BASE_DIR + 'sgns.renmin.bigram-char'#单词->300维embedding
MIN_SEQUENCE_LENGTH = 5  #最小序列长度5
MAX_SEQUENCE_LENGTH = 40 #最大序列长度40
MAX_NB_WORDS = 93250
EMBEDDING_DIM = 300 #embedding维度300

texts = [] #通过列表来存储句子
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = f.readline()
    while reader: #取出句子,存入texts
        if (len(reader.split()) <= MAX_SEQUENCE_LENGTH) and (len(reader.split()) >= MIN_SEQUENCE_LENGTH):
            texts.append(reader)
        reader = f.readline()
f.close()

n_sents = len(texts)
print('Found %s texts in train.txt' % n_sents) #训练用句子个数

Found 106508 texts in train.txt


### 文本预处理
使用Keras的tokenizer和text_to_sequences函数预处理文本

In [3]:
tokenizer = Tokenizer(MAX_NB_WORDS+1, oov_token='unk') #Tokenizer是一个用于向量化文本，或将文本转换为序列（即单词在字典中的下标构成的列表，从1算起）的类
tokenizer.fit_on_texts(texts)
print('Found %s unique tokens' % len(tokenizer.word_index))

## **关键步骤** 若不能正常工作，丢弃OOV_Token
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= MAX_NB_WORDS} # <= 从1开始
#tokenizer.word_index[tokenizer.oov_token] = MAX_NB_WORDS + 1
word_index = tokenizer.word_index #word到index的字典
index2word = {v: k for k, v in word_index.items()} #index到word的字典

sequences = tokenizer.texts_to_sequences(texts)#序列的列表，列表中每个序列对应于一段输入文本
data_1 = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) #序列全部填充到25维，尾补0
print('Shape of data tensor:', data_1.shape)
NB_WORDS = (min(tokenizer.num_words, len(word_index))+1) #+1 for zero padding 
print('NB_WORDS:', NB_WORDS)

data_val = data_1[100000:106500]
data_train = data_1[:100000]


Found 93254 unique tokens
Shape of data tensor: (106508, 40)
NB_WORDS: 93251


In [4]:
print(word_index['unk'])

1


In [5]:
print(index2word[93250])

外八庙


### Word embeddings
使用预训练的Glove word embeddings。创建一个矩阵，在词汇表中为每个单词对应一个embedding，然后我们将这个矩阵作为权重传递给我们模型的embedding layer 

In [8]:
embeddings_index = {}

#取出word及其对应的embeddings，存入字典embeddings_index
with codecs.open(GLOVE_EMBEDDING, encoding='utf-8') as f:
    line = f.readline()
    line = f.readline()
    while line: #取出句子,存入texts
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        line = f.readline()
f.close()
print('Found %s word vectors.' % len(embeddings_index))


glove_embedding_matrix = np.zeros((NB_WORDS, EMBEDDING_DIM)) #申请0数组，(93251,300)
for word, i in word_index.items():
    if i < NB_WORDS+1: #+1 for 'unk' oov token 
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            glove_embedding_matrix[i] = embedding_vector
        else:
            # 在embeddings索引中找不到的单词，将是unk的embeddings
            #print('i=',i)
            glove_embedding_matrix[i] = embeddings_index.get('unk')
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))

Found 355776 word vectors.
i= 1
i= 4
i= 5
i= 217
i= 229
i= 242
i= 254
i= 269
i= 281
i= 305
i= 369
i= 424
i= 523
i= 527
i= 625
i= 662
i= 843
i= 891
i= 924
i= 1033
i= 1134
i= 1210
i= 1218
i= 1234
i= 1249
i= 1381
i= 1470
i= 1601
i= 1605
i= 1624
i= 1669
i= 1672
i= 1717
i= 1728
i= 1752
i= 1786
i= 1833
i= 1932
i= 1977
i= 1983
i= 1993
i= 2034
i= 2039
i= 2051
i= 2090
i= 2093
i= 2102
i= 2104
i= 2150
i= 2151
i= 2167
i= 2169
i= 2192
i= 2219
i= 2222
i= 2290
i= 2358
i= 2377
i= 2388
i= 2392
i= 2428
i= 2439
i= 2470
i= 2473
i= 2506
i= 2520
i= 2521
i= 2526
i= 2573
i= 2586
i= 2621
i= 2641
i= 2643
i= 2652
i= 2685
i= 2692
i= 2712
i= 2714
i= 2718
i= 2720
i= 2727
i= 2729
i= 2743
i= 2744
i= 2752
i= 2807
i= 2809
i= 2824
i= 2839
i= 2855
i= 2878
i= 2934
i= 2967
i= 2986
i= 2995
i= 3006
i= 3018
i= 3022
i= 3035
i= 3037
i= 3045
i= 3075
i= 3122
i= 3123
i= 3124
i= 3173
i= 3193
i= 3225
i= 3227
i= 3263
i= 3290
i= 3332
i= 3351
i= 3370
i= 3441
i= 3516
i= 3521
i= 3550
i= 3561
i= 3562
i= 3565
i= 3576
i= 3577
i= 3638
i= 364

i= 18096
i= 18098
i= 18100
i= 18106
i= 18139
i= 18161
i= 18282
i= 18294
i= 18331
i= 18334
i= 18364
i= 18369
i= 18370
i= 18376
i= 18378
i= 18382
i= 18387
i= 18401
i= 18455
i= 18471
i= 18474
i= 18485
i= 18493
i= 18526
i= 18535
i= 18537
i= 18538
i= 18541
i= 18545
i= 18561
i= 18563
i= 18596
i= 18607
i= 18620
i= 18683
i= 18690
i= 18691
i= 18692
i= 18709
i= 18734
i= 18735
i= 18754
i= 18761
i= 18775
i= 18868
i= 18903
i= 18953
i= 18979
i= 19065
i= 19070
i= 19079
i= 19099
i= 19122
i= 19141
i= 19163
i= 19206
i= 19217
i= 19237
i= 19243
i= 19245
i= 19246
i= 19250
i= 19252
i= 19270
i= 19274
i= 19280
i= 19286
i= 19287
i= 19290
i= 19293
i= 19294
i= 19295
i= 19302
i= 19303
i= 19305
i= 19306
i= 19309
i= 19316
i= 19318
i= 19325
i= 19332
i= 19333
i= 19334
i= 19336
i= 19337
i= 19338
i= 19345
i= 19346
i= 19347
i= 19349
i= 19359
i= 19360
i= 19363
i= 19366
i= 19367
i= 19372
i= 19373
i= 19377
i= 19382
i= 19383
i= 19384
i= 19388
i= 19393
i= 19397
i= 19401
i= 19402
i= 19403
i= 19405
i= 19410
i= 19411
i= 19413
i

i= 26311
i= 26342
i= 26344
i= 26386
i= 26403
i= 26408
i= 26411
i= 26426
i= 26445
i= 26453
i= 26482
i= 26484
i= 26486
i= 26503
i= 26515
i= 26557
i= 26570
i= 26602
i= 26632
i= 26639
i= 26660
i= 26672
i= 26676
i= 26681
i= 26714
i= 26732
i= 26750
i= 26756
i= 26789
i= 26796
i= 26851
i= 26856
i= 26863
i= 26871
i= 26876
i= 26913
i= 26915
i= 26925
i= 26929
i= 26934
i= 26936
i= 26949
i= 26981
i= 27012
i= 27070
i= 27074
i= 27102
i= 27106
i= 27114
i= 27122
i= 27124
i= 27128
i= 27131
i= 27135
i= 27160
i= 27161
i= 27172
i= 27182
i= 27195
i= 27196
i= 27209
i= 27215
i= 27218
i= 27231
i= 27275
i= 27277
i= 27282
i= 27284
i= 27285
i= 27286
i= 27287
i= 27290
i= 27292
i= 27295
i= 27304
i= 27306
i= 27312
i= 27314
i= 27315
i= 27318
i= 27329
i= 27335
i= 27339
i= 27353
i= 27365
i= 27378
i= 27392
i= 27393
i= 27397
i= 27398
i= 27400
i= 27402
i= 27405
i= 27421
i= 27423
i= 27424
i= 27425
i= 27426
i= 27427
i= 27431
i= 27433
i= 27434
i= 27442
i= 27443
i= 27444
i= 27445
i= 27453
i= 27459
i= 27462
i= 27466
i= 27468
i

i= 33216
i= 33223
i= 33225
i= 33226
i= 33227
i= 33228
i= 33231
i= 33232
i= 33234
i= 33235
i= 33236
i= 33238
i= 33239
i= 33241
i= 33242
i= 33252
i= 33265
i= 33267
i= 33274
i= 33275
i= 33276
i= 33278
i= 33280
i= 33281
i= 33282
i= 33284
i= 33285
i= 33290
i= 33294
i= 33296
i= 33297
i= 33299
i= 33301
i= 33302
i= 33310
i= 33311
i= 33315
i= 33316
i= 33317
i= 33321
i= 33323
i= 33325
i= 33327
i= 33336
i= 33341
i= 33351
i= 33353
i= 33355
i= 33357
i= 33360
i= 33364
i= 33365
i= 33368
i= 33374
i= 33375
i= 33377
i= 33379
i= 33380
i= 33386
i= 33387
i= 33389
i= 33391
i= 33394
i= 33397
i= 33398
i= 33409
i= 33416
i= 33419
i= 33426
i= 33429
i= 33432
i= 33434
i= 33436
i= 33440
i= 33441
i= 33442
i= 33443
i= 33444
i= 33446
i= 33447
i= 33448
i= 33450
i= 33452
i= 33453
i= 33454
i= 33458
i= 33461
i= 33463
i= 33464
i= 33470
i= 33471
i= 33472
i= 33473
i= 33481
i= 33489
i= 33494
i= 33495
i= 33506
i= 33512
i= 33528
i= 33529
i= 33531
i= 33533
i= 33541
i= 33543
i= 33547
i= 33549
i= 33559
i= 33560
i= 33561
i= 33562
i

i= 38785
i= 38786
i= 38790
i= 38791
i= 38798
i= 38801
i= 38802
i= 38804
i= 38806
i= 38809
i= 38821
i= 38824
i= 38830
i= 38833
i= 38834
i= 38848
i= 38850
i= 38854
i= 38858
i= 38866
i= 38876
i= 38902
i= 38908
i= 38915
i= 38918
i= 38920
i= 38921
i= 38923
i= 38924
i= 38927
i= 38937
i= 38942
i= 38946
i= 38963
i= 38964
i= 38965
i= 38967
i= 38996
i= 38998
i= 39001
i= 39003
i= 39005
i= 39007
i= 39009
i= 39010
i= 39012
i= 39013
i= 39019
i= 39047
i= 39055
i= 39059
i= 39085
i= 39086
i= 39088
i= 39092
i= 39095
i= 39101
i= 39107
i= 39108
i= 39110
i= 39113
i= 39114
i= 39123
i= 39125
i= 39132
i= 39134
i= 39143
i= 39147
i= 39149
i= 39161
i= 39165
i= 39172
i= 39183
i= 39195
i= 39200
i= 39207
i= 39210
i= 39227
i= 39233
i= 39245
i= 39247
i= 39257
i= 39279
i= 39281
i= 39288
i= 39299
i= 39301
i= 39302
i= 39304
i= 39307
i= 39310
i= 39325
i= 39328
i= 39344
i= 39370
i= 39435
i= 39449
i= 39451
i= 39461
i= 39470
i= 39471
i= 39478
i= 39479
i= 39480
i= 39481
i= 39482
i= 39484
i= 39487
i= 39495
i= 39497
i= 39498
i

...

i= 84351
i= 84352
i= 84353
i= 84354
i= 84355
i= 84356
i= 84357
i= 84358
i= 84360
i= 84361
i= 84364
i= 84366
i= 84367
i= 84370
i= 84374
i= 84375
i= 84378
i= 84379
i= 84381
i= 84382
i= 84383
i= 84384
i= 84385
i= 84386
i= 84388
i= 84389
i= 84390
i= 84392
i= 84395
i= 84396
i= 84398
i= 84399
i= 84400
i= 84401
i= 84402
i= 84405
i= 84406
i= 84407
i= 84409
i= 84410
i= 84411
i= 84413
i= 84415
i= 84416
i= 84417
i= 84418
i= 84419
i= 84420
i= 84421
i= 84422
i= 84424
i= 84426
i= 84427
i= 84428
i= 84429
i= 84430
i= 84431
i= 84432
i= 84433
i= 84434
i= 84437
i= 84440
i= 84441
i= 84442
i= 84443
i= 84445
i= 84446
i= 84447
i= 84449
i= 84450
i= 84451
i= 84452
i= 84455
i= 84456
i= 84457
i= 84458
i= 84459
i= 84460
i= 84461
i= 84462
i= 84463
i= 84465
i= 84466
i= 84467
i= 84469
i= 84470
i= 84471
i= 84473
i= 84477
i= 84478
i= 84479
i= 84480
i= 84481
i= 84482
i= 84483
i= 84484
i= 84485
i= 84486
i= 84487
i= 84488
i= 84491
i= 84493
i= 84495
i= 84496
i= 84497
i= 84498
i= 84499
i= 84500
i= 84503
i= 84504
i= 84506
i

i= 85620
i= 85621
i= 85623
i= 85624
i= 85625
i= 85627
i= 85628
i= 85629
i= 85630
i= 85631
i= 85632
i= 85634
i= 85636
i= 85637
i= 85639
i= 85640
i= 85641
i= 85642
i= 85643
i= 85644
i= 85645
i= 85646
i= 85647
i= 85648
i= 85650
i= 85652
i= 85653
i= 85654
i= 85655
i= 85660
i= 85661
i= 85663
i= 85664
i= 85665
i= 85668
i= 85670
i= 85671
i= 85672
i= 85673
i= 85674
i= 85676
i= 85677
i= 85679
i= 85680
i= 85684
i= 85685
i= 85686
i= 85689
i= 85690
i= 85691
i= 85692
i= 85693
i= 85694
i= 85695
i= 85696
i= 85698
i= 85699
i= 85700
i= 85703
i= 85704
i= 85705
i= 85706
i= 85707
i= 85708
i= 85709
i= 85710
i= 85712
i= 85714
i= 85719
i= 85720
i= 85722
i= 85723
i= 85724
i= 85725
i= 85726
i= 85728
i= 85730
i= 85731
i= 85732
i= 85733
i= 85737
i= 85740
i= 85741
i= 85743
i= 85744
i= 85745
i= 85748
i= 85750
i= 85751
i= 85752
i= 85753
i= 85755
i= 85756
i= 85760
i= 85761
i= 85765
i= 85766
i= 85767
i= 85768
i= 85769
i= 85770
i= 85771
i= 85772
i= 85773
i= 85774
i= 85775
i= 85776
i= 85779
i= 85780
i= 85782
i= 85783
i

i= 88377
i= 88378
i= 88379
i= 88381
i= 88383
i= 88384
i= 88385
i= 88386
i= 88387
i= 88388
i= 88389
i= 88390
i= 88394
i= 88395
i= 88396
i= 88399
i= 88400
i= 88401
i= 88403
i= 88405
i= 88407
i= 88408
i= 88410
i= 88414
i= 88415
i= 88417
i= 88419
i= 88420
i= 88421
i= 88422
i= 88423
i= 88424
i= 88425
i= 88426
i= 88429
i= 88430
i= 88431
i= 88432
i= 88433
i= 88434
i= 88435
i= 88436
i= 88439
i= 88440
i= 88441
i= 88442
i= 88443
i= 88447
i= 88448
i= 88449
i= 88450
i= 88451
i= 88452
i= 88453
i= 88454
i= 88456
i= 88457
i= 88458
i= 88459
i= 88460
i= 88461
i= 88462
i= 88463
i= 88464
i= 88465
i= 88467
i= 88468
i= 88469
i= 88470
i= 88471
i= 88472
i= 88476
i= 88477
i= 88480
i= 88481
i= 88482
i= 88483
i= 88486
i= 88487
i= 88488
i= 88489
i= 88490
i= 88491
i= 88494
i= 88496
i= 88497
i= 88498
i= 88499
i= 88501
i= 88503
i= 88505
i= 88506
i= 88514
i= 88515
i= 88518
i= 88520
i= 88521
i= 88522
i= 88525
i= 88526
i= 88528
i= 88530
i= 88533
i= 88535
i= 88537
i= 88539
i= 88545
i= 88547
i= 88548
i= 88550
i= 88557
i

i= 89920
i= 89922
i= 89923
i= 89924
i= 89927
i= 89928
i= 89929
i= 89931
i= 89932
i= 89933
i= 89936
i= 89937
i= 89938
i= 89939
i= 89940
i= 89943
i= 89944
i= 89946
i= 89947
i= 89948
i= 89952
i= 89959
i= 89960
i= 89961
i= 89962
i= 89965
i= 89967
i= 89968
i= 89970
i= 89971
i= 89972
i= 89979
i= 89981
i= 89983
i= 89984
i= 89985
i= 89986
i= 89988
i= 89989
i= 89990
i= 89991
i= 89992
i= 89993
i= 89996
i= 89998
i= 90000
i= 90002
i= 90004
i= 90005
i= 90006
i= 90007
i= 90008
i= 90009
i= 90010
i= 90011
i= 90013
i= 90014
i= 90015
i= 90016
i= 90017
i= 90018
i= 90019
i= 90021
i= 90022
i= 90026
i= 90027
i= 90028
i= 90030
i= 90031
i= 90032
i= 90033
i= 90034
i= 90035
i= 90036
i= 90037
i= 90038
i= 90039
i= 90040
i= 90041
i= 90042
i= 90043
i= 90046
i= 90047
i= 90048
i= 90049
i= 90050
i= 90051
i= 90053
i= 90055
i= 90057
i= 90058
i= 90060
i= 90061
i= 90063
i= 90065
i= 90070
i= 90071
i= 90072
i= 90073
i= 90074
i= 90075
i= 90077
i= 90082
i= 90083
i= 90087
i= 90088
i= 90090
i= 90091
i= 90092
i= 90093
i= 90094
i

i= 91723
i= 91725
i= 91727
i= 91728
i= 91729
i= 91731
i= 91732
i= 91734
i= 91739
i= 91741
i= 91742
i= 91743
i= 91748
i= 91760
i= 91763
i= 91766
i= 91770
i= 91774
i= 91776
i= 91782
i= 91783
i= 91786
i= 91787
i= 91794
i= 91802
i= 91806
i= 91811
i= 91820
i= 91823
i= 91825
i= 91829
i= 91830
i= 91832
i= 91833
i= 91834
i= 91835
i= 91836
i= 91837
i= 91838
i= 91840
i= 91842
i= 91843
i= 91845
i= 91846
i= 91847
i= 91848
i= 91849
i= 91851
i= 91852
i= 91854
i= 91855
i= 91856
i= 91857
i= 91858
i= 91859
i= 91860
i= 91861
i= 91862
i= 91863
i= 91864
i= 91865
i= 91866
i= 91867
i= 91868
i= 91869
i= 91870
i= 91872
i= 91873
i= 91874
i= 91875
i= 91876
i= 91877
i= 91878
i= 91881
i= 91882
i= 91883
i= 91885
i= 91888
i= 91889
i= 91890
i= 91891
i= 91892
i= 91896
i= 91898
i= 91899
i= 91900
i= 91902
i= 91903
i= 91904
i= 91905
i= 91906
i= 91909
i= 91915
i= 91919
i= 91920
i= 91937
i= 91942
i= 91943
i= 91944
i= 91945
i= 91948
i= 91949
i= 91953
i= 91954
i= 91960
i= 91976
i= 91977
i= 91978
i= 91979
i= 91983
i= 91985
i

### VAE 模型
模型基于seq2seq架构，包含双向LSTM编码器和LSTM解码器。
通过 RepeatVector（max_len）函数，将每个时间步的潜在表示作为输入提供给解码器decoder。为了避免标签的独热码表示，我们使用tf.contrib.seq2seq.sequence_loss函数，它只需要单词索引作为标签（与embedding矩阵的输入相同）并在内部计算最终的softmax（所以 模型以具有线性激活的dense层结束）。 
可选地，“sequence_loss”允许使用采样的softmax，这有助于处理大型词汇表（例如，具有50k字词汇），但在此没有使用。这里使用的解码器与文中实现的解码器不同; 不是将context vector作为解码器的初始状态和预测的单词作为输入，而是在每个时间步处输入潜在表示z作为输入。

In [9]:
batch_size = 100
max_len = MAX_SEQUENCE_LENGTH
emb_dim = EMBEDDING_DIM
latent_dim = 64
intermediate_dim = 256
epsilon_std = 1.0
kl_weight = 0.01
num_sampled=500
act = ELU()


x = Input(shape=(max_len,)) #输入是按批量的40维向量(句子)
x_embed = Embedding(NB_WORDS, emb_dim, weights=[glove_embedding_matrix], input_length=max_len, trainable=False)(x)
h = Bidirectional(LSTM(intermediate_dim, return_sequences=False, recurrent_dropout=0.2), merge_mode='concat')(x_embed)
#h = Bidirectional(LSTM(intermediate_dim, return_sequences=False), merge_mode='concat')(h)
#h = Dropout(0.2)(h)
#h = Dense(intermediate_dim, activation='linear')(h)
#h = act(h)
#h = Dropout(0.2)(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
# 分别实例化这些层，以便以后重用
repeated_context = RepeatVector(max_len)
decoder_h = LSTM(intermediate_dim, return_sequences=True, recurrent_dropout=0.2)
decoder_mean = Dense(NB_WORDS, activation='linear')#softmax is applied in the seq2seqloss by tf #TimeDistributed()
h_decoded = decoder_h(repeated_context(z))
x_decoded_mean = decoder_mean(h_decoded)


# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)

#Sampled softmax
#logits = tf.constant(np.random.randn(batch_size, max_len, NB_WORDS), tf.float32)
#targets = tf.constant(np.random.randint(NB_WORDS, size=(batch_size, max_len)), tf.int32)
#proj_w = tf.constant(np.random.randn(NB_WORDS, NB_WORDS), tf.float32)
#proj_b = tf.constant(np.zeros(NB_WORDS), tf.float32)
#
#def _sampled_loss(labels, logits):
#    labels = tf.cast(labels, tf.int64)
#    labels = tf.reshape(labels, [-1, 1])
#    logits = tf.cast(logits, tf.float32)
#    return tf.cast(
#                    tf.nn.sampled_softmax_loss(
#                        proj_w,
#                        proj_b,
#                        labels,
#                        logits,
#                        num_sampled=num_sampled,
#                        num_classes=NB_WORDS),
#                    tf.float32)
#softmax_loss_f = _sampled_loss


# 用于计算VAE损失的自定义层
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        self.target_weights = tf.constant(np.ones((batch_size, max_len)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        #xent_loss = K.sum(metrics.categorical_crossentropy(x, x_decoded_mean), axis=-1)
        labels = tf.cast(x, tf.int32)
        xent_loss = K.sum(tf.contrib.seq2seq.sequence_loss(x_decoded_mean, labels, 
                                                     weights=self.target_weights,
                                                     average_across_timesteps=False,
                                                     average_across_batch=False), axis=-1)#,
                                                     #softmax_loss_function=softmax_loss_f), axis=-1)#,
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        xent_loss = K.mean(xent_loss)
        kl_loss = K.mean(kl_loss)
        return K.mean(xent_loss + kl_weight * kl_loss)
    
    #编写一个call方法，来实现自定义层
    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)
    
def kl_loss(x, x_decoded_mean):
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    kl_loss = kl_weight * kl_loss
    return kl_loss

loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])
opt = Adam(lr=0.01) 
vae.compile(optimizer='adam', loss=[zero_loss], metrics=[kl_loss])
vae.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(?, 40) (?, 40, 93251)

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 40, 300)      27975300    input_1[0][0]                    
___________________________________________________________

### 模型训练
通过keras.fit()训练100epochs。对于验证数据，传递相同的数组两次，因为此模型的输入和标签相同。
如果不使用“tf.contrib.seq2seq.sequence_loss”（或其他类似的函数），
将必须作为标签传递单词的one-hot码高维度序列(batch_size，seq_len，vocab_size)消耗大量内存。

In [10]:
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + model_name + ".h5" 
    directory = os.path.dirname(filepath)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)
    return checkpointer

checkpointer = create_model_checkpoint('models', 'vae_seq2seq_test_very_high_std')



vae.fit(data_train, data_train,
     shuffle=True,
     epochs=100,
     batch_size=batch_size,
     validation_data=(data_val, data_val), callbacks=[checkpointer])

#print(K.eval(vae.optimizer.lr))
#K.set_value(vae.optimizer.lr, 0.01)

vae.save('models/vae_lstm.h5')
#vae.load_weights('models/vae_lstm.h5')

Instructions for updating:
Use tf.cast instead.
Train on 100000 samples, validate on 6500 samples
Epoch 1/100


KeyboardInterrupt: 